# Retrosheet Baseball Data -- Wrangle Data


**Baseball Notebooks**  
1. Download and unzipped the Lahman and Retrosheet data.
2. Described helper functions used by several notebooks.
3. Baseball Data Organization and Data Dictionary
3. Lahman data was wrangled and persisted.
4. Retrosheet Play by Play data was parsed, collected into 2 DataFrames, and persisted.
6. This notebook.

Wrangle the Retrosheet data in preparation for data analysis.

This notebook is designed to be used with Jupyter Lab and the Table of Contents extension.  
https://github.com/jupyterlab/jupyterlab-toc

## Data Wrangling

Four Retrosheet data objects will be wrangled and 2 Data Dictionaries will be uncovered.

**Data Dictionaries**  
Created by parsing the C source code for the parsers.  Could also be created by using the '-d' and '-n' parser options and combining the results.
1. cwdaily
2. cwgame

**Data Objects**  
1. cwdaily produces a single csv file that will be used to create 1 DataFrame:
  * player_game: batting/pitching/fielding attributes per player per game
2. cwgame produces a single csv file that will be used to create 2 DataFrames:
  * game: game attributes (time, attendance, etc.)
  * team_game: batting/pitching/fielding attributes per team per game
3. player "lookup" table scraped from Retrosheet web site
4. park (stadium) "lookup" table scraped from Retorsheet web site

All of the above will be persisted as csv files with column types.  The following notebook will persist these csv files as Postgres tables.

## Repeatable Research
All data processing should be documented so that others can repeat the results.  This includes every step from downloading the data through analysis.

### Directories for Data Processing

* ~/data/retrosheet/raw -- event files downloaded from Retrosheet
* ~/data/retrosheet/parsed -- results of running 2 parsers on the event files
* ~/data/retrosheet/df_csv -- collect the parsed files into dataframes and save these to csv
* ~/data/retrosheet/wrangled -- prepare the data for analsyis and save to csv
* ~/data/retrosheet/src -- optional directory to hold parser source code

In [1]:
import pandas as pd
import numpy as np
import os
import re
from pathlib import Path

from IPython.display import HTML, display

In [2]:
# see Baseball Notebook #2
import helper_functions as bb

In [3]:
# create path objects
home = Path.home()
retrosheet = home.joinpath('data/retrosheet')
p_raw = retrosheet.joinpath('raw')
p_wrangled = retrosheet.joinpath('wrangled')

p_parsed = retrosheet.joinpath('parsed')
p_collected = retrosheet.joinpath('collected')
p_src = retrosheet.joinpath('src')

## Stats per Player per Game: Data Dictionary
This section is optional.  It may be helpful to understanding the data.

cwdaily can be run with the '-n' flag to have it output field names, and separately it can be run with the '-d' flag to have it output field descriptions.  The field names and field descriptions could then be put together to form a data dictionary.

Partly as an exercise in regular expressions, a different approach was used here.  The C source code was parsed to produce the data dictionary.  It is not necessary to understand the RegEx code for parsing the C struct that has the field name and the field descriptions.

In [140]:
# cd to dir with cwdaily.c
src = retrosheet.joinpath('src')
os.chdir(src)

In [141]:
def parse_c_source(filename, struct='field_data'):
    """Extract field name to field description from parser's C source code"""
    dd = {}
    with open(filename, 'r') as cwdaily:
        # to account for patterns across lines, read entire source code
        source = cwdaily.read()
    
        # get the single (multiline) C statement that has field descriptions
        pattern = r'(static\s+field_struct\s+' + struct + r'.*?;)'
        match = re.search(pattern, source, flags=re.DOTALL | re.MULTILINE)
    
        if match:
            pattern = r'{.*?"(.*?)".*?"(.*?)".*?}'
            for m in re.finditer(pattern, match.group(1), 
                                 flags=re.DOTALL | re.MULTILINE):
                if m:
                    if len(m.group(2).split(':')) == 2:
                        desc = m.group(2).split(':')[1].strip()
                    else:
                        desc = m.group(2).strip()
                    dd[m.group(1).lower()] = desc   
    return dd

In [142]:
cwdaily_fields = parse_c_source('cwdaily.c')        

In [143]:
# here is the explanation of each field, as scraped from the C source code
cwdaily_fields

{'game_id': 'game id',
 'game_dt': 'date',
 'game_ct': 'game number (0 = no double header)',
 'appear_dt': 'apperance date',
 'team_id': 'team id',
 'player_id': 'player id',
 'b_g': 'games played',
 'b_pa': 'plate appearances',
 'b_ab': 'at bats',
 'b_r': 'runs',
 'b_h': 'hits',
 'b_2b': 'doubles',
 'b_3b': 'triples',
 'b_hr': 'home runs',
 'b_rbi': 'runs batted in',
 'b_bb': 'walks',
 'b_ibb': 'intentional walks',
 'b_so': 'strikeouts',
 'b_gdp': 'grounded into DP',
 'b_hp': 'hit by pitch',
 'b_sh': 'sacrifice hits',
 'b_sf': 'sacrifice flies',
 'b_sb': 'stolen bases',
 'b_cs': 'caught stealing',
 'b_xi': 'reached on interference',
 'p_g': 'games pitched',
 'p_gs': 'games started',
 'p_cg': 'complete games',
 'p_sho': 'shutouts',
 'p_gf': 'games finished',
 'p_w': 'wins',
 'p_l': 'losses',
 'p_sv': 'saves',
 'p_out': 'outs recorded (innings pitched times 3)',
 'p_tbf': 'batters faced',
 'p_ab': 'at bats',
 'p_r': 'runs allowed',
 'p_er': 'earned runs allowed',
 'p_h': 'hits allowed',

### Data Dictionary Notes

Fields with the prefix b_ represent batter attributes.  
Fields with the prefix p_ represent pitcher attributes.  
Fields with the prefix f_ represent fielder attributes.  

Note that team_id is the team_id of the player.

game_id is a string consisting of 3 fields:
```
0:3 Home TEAM_ID  
3:11 YYYYMMDD  
12   Game Count
```

Game Count is:
* 0 for single game
* 1 for 1st game of double header
* 2 for 2nd game of double header

### Persist Stats per Player per Game Data Dictionary Fields

In [144]:
os.chdir(p_wrangled)

# index=[0] is required for dictionary of scalar values
cwdaily_fields_df = pd.DataFrame(cwdaily_fields, index=[0])
cwdaily_fields_df.to_csv('cwdaily_fields.csv', index=False)

## Stats per Game: Data Dictionary
This section is optional.  It may be helpful to understanding the data.

There is a field-name to field-description mapping provided on the following web page:  
http://chadwick.sourceforge.net/doc/cwgame.html

Alternatively, as with cwdaily, cwgame can be run with the '-n' flag to have it output field names, and separately it can be run with the '-d' flag to have it output field descriptions. The field names and field descriptions could then be put together to form a data dictionary.

Alternatively, as a regular expression parser was already built, it can be used to parse cwdaily to retrieve the data dictionary.  That is the approach taken here.

Note: the codes for some of the \_CD fields are only specified on the above web page.

In [145]:
os.chdir(src)
game_reg_fields = parse_c_source('cwgame.c')
game_ext_fields = parse_c_source('cwgame.c', 'ext_field_data')
cwgame_fields = {**game_reg_fields, **game_ext_fields}

In [146]:
# there are 84 regular fields and 95 extended fields
len(game_reg_fields), len(game_ext_fields), len(game_fields)

(84, 95, 179)

In [147]:
cwgame_fields

{'game_id': 'game id',
 'game_dt': 'date',
 'game_ct': 'game number (0 = no double header)',
 'game_dy': 'day of week',
 'start_game_tm': 'start time',
 'dh_fl': 'DH used flag',
 'daynight_park_cd': 'day/night flag',
 'away_team_id': 'visiting team',
 'home_team_id': 'home team',
 'park_id': 'game site',
 'away_start_pit_id': 'vis. starting pitcher',
 'home_start_pit_id': 'home starting pitcher',
 'base4_ump_id': 'home plate umpire',
 'base1_ump_id': 'first base umpire',
 'base2_ump_id': 'second base umpire',
 'base3_ump_id': 'third base umpire',
 'lf_ump_id': 'left field umpire',
 'rf_ump_id': 'right field umpire',
 'attend_park_ct': 'attendance',
 'scorer_record_id': 'PS scorer',
 'translator_record_id': 'translator',
 'inputter_record_id': 'inputter',
 'input_record_ts': 'input time',
 'edit_record_ts': 'edit time',
 'method_record_cd': 'how scored',
 'pitches_record_cd': 'pitches entered?',
 'temp_park_ct': 'temperature',
 'wind_direction_park_cd': 'wind direction',
 'wind_speed_pa

### Data Dictionary Note
dh_fl: Designated Hitter Flag, 'T' if DH in use, else 'F'  
daynight_park_cd: 'N' for night, 'D' for day  
gw_rbi_bat_id: Player ID for batter who got Game Winning RBI  

### Persist Stats per Game Data Dictionary Fields

In [148]:
os.chdir(p_wrangled)

# index=[0] is required for dictionary of scalar values
cwgame_fields_df = pd.DataFrame(cwgame_fields, index=[0])
cwgame_fields_df.to_csv('cwgame_fields.csv', index=False)

## 1. Wrangle Stats per Player Per Game Data

### Manual Data Verification

For odd data, such as whether or not a the first game of a double header was in one stadium, and the second game was in a different stadium, [Baseball-Reference](https://www.baseball-reference.com) is helpful.

Baseball-reference uses the data from Retrosheet, and presents it in an easy to read form for people. On rare occasions it may incorrectly present the event data, but it is a useful tool.

Baseball-reference does not offer already parsed data for data analysis.

The following method takes a game_id and converts it to a baseball-reference url for researching more about a particular game.

In [13]:
from IPython.display import HTML, display
def game_id_to_url(game_id):
    home = game_id[:3]
    url = 'https://www.baseball-reference.com/boxes/' + home + '/' + game_id + '.shtml'
    display(HTML(f'<a href="{url}">{game_id}</a>'))

In [14]:
# Click on the generated link to get a url for detailed game information.
game_id_to_url('NYA200806271')

As per the above link, the first game of the double header was in Yankee Stadium and the second game, on the same day, was in Shea Stadium.

In [15]:
# another oddity, the public was not allowed to attend
game_id_to_url('BAL201504290')

### Read in Stats per Player per Game Data 
This is the data created by running the cwdaily parser in the previous notebook.

In [16]:
# read in the parsed player_game data
os.chdir(p_collected)
player_game = bb.from_csv_with_types('player_game.csv.gz')

In [17]:
player_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3549700 entries, 0 to 3549699
Columns: 117 entries, game_id to f_rf_tp
dtypes: object(3), uint32(2), uint8(112)
memory usage: 487.5+ MB


In [18]:
player_game.shape

(3549700, 117)

In [19]:
player_game.tail(3)

,game_id,game_dt,game_ct,appear_dt,team_id,player_id,b_g,b_pa,b_ab,b_r,...,f_cf_e,f_cf_dp,f_cf_tp,f_rf_g,f_rf_out,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp
3549697,WAS201809260,20180926,0,20180926,WAS,rodrj005,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3549698,WAS201809260,20180926,0,20180926,WAS,eatoa002,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3549699,WAS201809260,20180926,0,20180926,WAS,glovk001,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Remove Column with same Data as another Column

In [20]:
(player_game['game_dt'] == player_game['appear_dt']).all()

True

In [21]:
player_game = player_game.drop('appear_dt', axis=1)

In [22]:
# the primary key is (game_id, PLAYER_ID), verify no dups
dups = player_game.duplicated(subset=['game_id', 'player_id'], keep=False)
player_game[dups]

,game_id,game_dt,game_ct,team_id,player_id,b_g,b_pa,b_ab,b_r,b_h,...,f_cf_e,f_cf_dp,f_cf_tp,f_rf_g,f_rf_out,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp
3418636,BOS201708250,20170825,0,BOS,younc004,1,3,3,0,1,...,0,0,0,0,0,0,0,0,0,0
3418638,BOS201708250,20170825,0,BOS,younc004,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# check this game manually by clicking on the generated link
dup_id = player_game.loc[dups, 'game_id'].values[0]
game_id_to_url(dup_id)

### Data Correction

Checking the box score via the above link, shows 2 entries for Young for the same game, one as a pinch-hitter and one as the designated-hitter.  It would appear that both entries are correct and that the data should be summed.

In [24]:
# get the index labels of the duplicated rows
idx1, idx2 = player_game[dups].index.values
idx1, idx2

(3418636, 3418638)

In [25]:
# identifier columns
id_columns = player_game.columns[:5]
id_columns

Index(['game_id', 'game_dt', 'game_ct', 'team_id', 'player_id'], dtype='object')

In [26]:
# column 5 is b_g, or games appeared in, leave this alone
# attributes to sum from the box score
stat_columns = player_game.columns[6:]
stat_columns

Index(['b_pa', 'b_ab', 'b_r', 'b_h', 'b_2b', 'b_3b', 'b_hr', 'b_rbi', 'b_bb',
       'b_ibb',
       ...
       'f_cf_e', 'f_cf_dp', 'f_cf_tp', 'f_rf_g', 'f_rf_out', 'f_rf_po',
       'f_rf_a', 'f_rf_e', 'f_rf_dp', 'f_rf_tp'],
      dtype='object', length=110)

In [27]:
# id columns match (as per df.duplicated() above)
player_game.loc[[idx1,idx2], id_columns]

,game_id,game_dt,game_ct,team_id,player_id
3418636,BOS201708250,20170825,0,BOS,younc004
3418638,BOS201708250,20170825,0,BOS,younc004


In [28]:
# game data
player_game.loc[[idx1,idx2], stat_columns]

,b_pa,b_ab,b_r,b_h,b_2b,b_3b,b_hr,b_rbi,b_bb,b_ibb,...,f_cf_e,f_cf_dp,f_cf_tp,f_rf_g,f_rf_out,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp
3418636,3,3,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3418638,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# add stats for the two rows
player_game.loc[idx1, stat_columns] += player_game.loc[idx2, stat_columns]

# remove duplicate row
player_game = player_game.drop(idx2)

In [30]:
# the primary key is (GAME_ID, PLAYER_ID), verify no dups
bb.is_unique(player_game, ['game_id', 'player_id'])

True

### Fielding Data

There are 65 fielding columns.  This is too much detail.  The fields that will be kept, per player per game are:
* out -- number of outs (summed over all positions) -- time playing defense measured in outs
* po -- put outs (summed over all positions)
* a -- assists  (summed over all positions)
* e -- errors  (summed over all positions)
* pb -- passed balls (only applies to catchers)
* xi -- runner reached based due catchers interference (only applies to catchers)

In [31]:
import re
cols = player_game.columns.copy()

# get all the field columns
f_cols = [col for col in cols if col.startswith('f_')]
len(f_cols)

65

In [32]:
cols = player_game.columns.copy()

# get all the put out columns
po_cols = [col for col in cols if col.endswith('_po')]
po_cols

['f_p_po',
 'f_c_po',
 'f_1b_po',
 'f_2b_po',
 'f_3b_po',
 'f_ss_po',
 'f_lf_po',
 'f_cf_po',
 'f_rf_po']

In [33]:
# a put out could be made at any of 9 positions, sum the put outs over the 9 positions
# note: sum by column is much faster than using apply per row
cols = po_cols.copy()

po = player_game[cols.pop()].copy()
while len(cols):
    po += player_game[cols.pop()]

In [34]:
cols = player_game.columns.copy()

# get all the assist columns
a_cols = [col for col in cols if col.endswith('_a')]
a_cols

['f_p_a',
 'f_c_a',
 'f_1b_a',
 'f_2b_a',
 'f_3b_a',
 'f_ss_a',
 'f_lf_a',
 'f_cf_a',
 'f_rf_a']

In [35]:
# sum the 9 assist columns into a single column
cols = a_cols.copy()

a = player_game[cols.pop()].copy()
while len(cols):
    a += player_game[cols.pop()]

In [36]:
cols = player_game.columns.copy()

# get all the error columns
e_cols = [col for col in cols if col.endswith('_e')]
e_cols

['f_p_e',
 'f_c_e',
 'f_1b_e',
 'f_2b_e',
 'f_3b_e',
 'f_ss_e',
 'f_lf_e',
 'f_cf_e',
 'f_rf_e']

In [37]:
# sum the 9 error columns, column by column (much faster than using apply on each row)
cols = e_cols.copy()

e = player_game[cols.pop()].copy()
while len(cols):
    e += player_game[cols.pop()]

In [38]:
cols = player_game.columns.copy()

# get all the error columns
out_cols = [col for col in cols if col.endswith('_out') and col.startswith('f_')]
out_cols

['f_p_out',
 'f_c_out',
 'f_1b_out',
 'f_2b_out',
 'f_3b_out',
 'f_ss_out',
 'f_lf_out',
 'f_cf_out',
 'f_rf_out']

In [39]:
# sum the 9 out columns
cols = out_cols.copy()

out = player_game[cols.pop()].copy()
while len(cols):
    out += player_game[cols.pop()]

In [40]:
# there is only 1 pb field: f_c_pb
# there is only 1 xi field: f_c_xi

In [41]:
# add the columns to player_game
player_game = player_game.assign(f_po=po)
player_game = player_game.assign(f_a=a)
player_game = player_game.assign(f_e=e)
player_game = player_game.assign(f_o=out)
player_game = player_game.assign(f_pb=player_game['f_c_pb'].copy())
player_game = player_game.assign(f_xi=player_game['f_c_xi'].copy())

# remove all 65 original fielding columns
player_game = player_game.drop(f_cols, axis = 1)

In [42]:
player_game.columns

Index(['game_id', 'game_dt', 'game_ct', 'team_id', 'player_id', 'b_g', 'b_pa',
       'b_ab', 'b_r', 'b_h', 'b_2b', 'b_3b', 'b_hr', 'b_rbi', 'b_bb', 'b_ibb',
       'b_so', 'b_gdp', 'b_hp', 'b_sh', 'b_sf', 'b_sb', 'b_cs', 'b_xi', 'p_g',
       'p_gs', 'p_cg', 'p_sho', 'p_gf', 'p_w', 'p_l', 'p_sv', 'p_out', 'p_tbf',
       'p_ab', 'p_r', 'p_er', 'p_h', 'p_2b', 'p_3b', 'p_hr', 'p_bb', 'p_ibb',
       'p_so', 'p_gdp', 'p_hp', 'p_sh', 'p_sf', 'p_xi', 'p_wp', 'p_bk', 'f_po',
       'f_a', 'f_e', 'f_o', 'f_pb', 'f_xi'],
      dtype='object')

### Add Lahman player_id
This will make joins to the Lahman data easier.

In [43]:
# get the People data for Lahman player_id
home = Path.home()
lahman = home.joinpath('data/lahman')
p_lahman_wrangled = lahman.joinpath('wrangled')

os.chdir(p_lahman_wrangled)
people = bb.from_csv_with_types('people.csv')

In [44]:
# add player_id from Lahman people
player_game = pd.merge(player_game, people[['player_id','retro_id']], 
                        left_on = 'player_id', right_on = 'retro_id',
                        suffixes=['', '_lahman'])

In [45]:
player_game = player_game.drop('retro_id', axis= 1)
player_game.columns

Index(['game_id', 'game_dt', 'game_ct', 'team_id', 'player_id', 'b_g', 'b_pa',
       'b_ab', 'b_r', 'b_h', 'b_2b', 'b_3b', 'b_hr', 'b_rbi', 'b_bb', 'b_ibb',
       'b_so', 'b_gdp', 'b_hp', 'b_sh', 'b_sf', 'b_sb', 'b_cs', 'b_xi', 'p_g',
       'p_gs', 'p_cg', 'p_sho', 'p_gf', 'p_w', 'p_l', 'p_sv', 'p_out', 'p_tbf',
       'p_ab', 'p_r', 'p_er', 'p_h', 'p_2b', 'p_3b', 'p_hr', 'p_bb', 'p_ibb',
       'p_so', 'p_gdp', 'p_hp', 'p_sh', 'p_sf', 'p_xi', 'p_wp', 'p_bk', 'f_po',
       'f_a', 'f_e', 'f_o', 'f_pb', 'f_xi', 'player_id_lahman'],
      dtype='object')

### Add Lahman team_id and year
This will make joins to the Lahman data easier.

The Lahman team_id can be found from the year and retrosheet team_id.

In [46]:
# get the teams data
os.chdir(p_lahman_wrangled)
teams = bb.from_csv_with_types('teams.csv')

In [47]:
# add year to player_game
player_game['year_id'] = player_game['game_dt'].apply(lambda x: x // 10000)

In [48]:
# add Lahman team_id from teams
player_game = pd.merge(player_game, teams[['team_id', 'year_id', 'team_id_retro']],
                       left_on = ['year_id', 'team_id'], 
                       right_on = ['year_id', 'team_id_retro'],
                       suffixes=['', '_lahman'])

In [49]:
player_game = player_game.drop('team_id_retro', axis= 1)
player_game.columns

Index(['game_id', 'game_dt', 'game_ct', 'team_id', 'player_id', 'b_g', 'b_pa',
       'b_ab', 'b_r', 'b_h', 'b_2b', 'b_3b', 'b_hr', 'b_rbi', 'b_bb', 'b_ibb',
       'b_so', 'b_gdp', 'b_hp', 'b_sh', 'b_sf', 'b_sb', 'b_cs', 'b_xi', 'p_g',
       'p_gs', 'p_cg', 'p_sho', 'p_gf', 'p_w', 'p_l', 'p_sv', 'p_out', 'p_tbf',
       'p_ab', 'p_r', 'p_er', 'p_h', 'p_2b', 'p_3b', 'p_hr', 'p_bb', 'p_ibb',
       'p_so', 'p_gdp', 'p_hp', 'p_sh', 'p_sf', 'p_xi', 'p_wp', 'p_bk', 'f_po',
       'f_a', 'f_e', 'f_o', 'f_pb', 'f_xi', 'player_id_lahman', 'year_id',
       'team_id_lahman'],
      dtype='object')

In [50]:
# put the key columns up front
player_game = bb.order_cols(player_game, 
                            ['game_id', 'player_id', 'team_id', 
                             'year_id', 'player_id_lahman', 
                             'team_id_lahman'])
player_game.columns

Index(['game_id', 'player_id', 'team_id', 'year_id', 'player_id_lahman',
       'team_id_lahman', 'game_dt', 'game_ct', 'b_g', 'b_pa', 'b_ab', 'b_r',
       'b_h', 'b_2b', 'b_3b', 'b_hr', 'b_rbi', 'b_bb', 'b_ibb', 'b_so',
       'b_gdp', 'b_hp', 'b_sh', 'b_sf', 'b_sb', 'b_cs', 'b_xi', 'p_g', 'p_gs',
       'p_cg', 'p_sho', 'p_gf', 'p_w', 'p_l', 'p_sv', 'p_out', 'p_tbf', 'p_ab',
       'p_r', 'p_er', 'p_h', 'p_2b', 'p_3b', 'p_hr', 'p_bb', 'p_ibb', 'p_so',
       'p_gdp', 'p_hp', 'p_sh', 'p_sf', 'p_xi', 'p_wp', 'p_bk', 'f_po', 'f_a',
       'f_e', 'f_o', 'f_pb', 'f_xi'],
      dtype='object')

In [51]:
# downcast the new columns
player_game = bb.optimize_df_dtypes(player_game)
player_game.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3549699 entries, 0 to 3549698
Data columns (total 60 columns):
game_id             object
player_id           object
team_id             object
year_id             uint16
player_id_lahman    object
team_id_lahman      object
game_dt             uint32
game_ct             uint8
b_g                 uint8
b_pa                uint8
b_ab                uint8
b_r                 uint8
b_h                 uint8
b_2b                uint8
b_3b                uint8
b_hr                uint8
b_rbi               uint8
b_bb                uint8
b_ibb               uint8
b_so                uint8
b_gdp               uint8
b_hp                uint8
b_sh                uint8
b_sf                uint8
b_sb                uint8
b_cs                uint8
b_xi                uint8
p_g                 uint8
p_gs                uint8
p_cg                uint8
p_sho               uint8
p_gf                uint8
p_w                 uint8
p_l                 ui

In [53]:
os.chdir(p_wrangled)
%time bb.to_csv_with_types(player_game, 'player_game.csv.gz')

CPU times: user 3min 44s, sys: 210 ms, total: 3min 44s
Wall time: 3min 44s


## 2. Wrangle Stats per Game Data

### Game Data Organization

There are "home" and "away" sets of fields.  Following the principles of tidy data, home and away will become a column, and there will be two records per game, one for each team. 

There are 3 types of data:
1. data specific to a game -- the 'rest' columns below
2. data specific to the home team for that game -- the 'home' columns below
3. data specific to the away team for that game -- the 'away' columns below

The attributes for the home team are identical to the attributes for the away team.

This suggests breaking this out into 2 tables.
1. key: (game_id) -- information specific to this particular game
2. key: (game_id, team_id) -- information for the specified team for the specified game

In [54]:
os.chdir(p_collected)
game = bb.from_csv_with_types('game.csv.gz')

In [55]:
home_cols = [col for col in game.columns if col.startswith('home')]
away_cols = [col for col in game.columns if col.startswith('away')]
rest_cols = [col for col in game.columns 
        if not col.startswith('home') and not col.startswith('away')]

In [56]:
home_cols[:4], away_cols[:4]

(['home_team_id', 'home_start_pit_id', 'home_score_ct', 'home_hits_ct'],
 ['away_team_id', 'away_start_pit_id', 'away_score_ct', 'away_hits_ct'])

In [57]:
# home column names are same as away column names (after removing 5 char prefix)
[col[5:] for col in home_cols] == [col[5:] for col in away_cols]

True

In [58]:
game_tidy = game[rest_cols].copy()
game_tidy.tail(3)

,game_id,game_dt,game_ct,game_dy,start_game_tm,dh_fl,daynight_park_cd,park_id,base4_ump_id,base1_ump_id,...,minutes_game_ct,inn_ct,win_pit_id,lose_pit_id,save_pit_id,gwrbi_bat_id,outs_ct,completion_tx,forfeit_tx,protest_tx
129843,WAS201809240,20180924,0,Monday,706,F,N,WAS11,herna901,porta901,...,189,9,millj006,alcas001,NaN,NaN,51,NaN,NaN,NaN
129844,WAS201809250,20180925,0,Tuesday,705,F,N,WAS11,porta901,whitc901,...,192,9,schem001,brigj002,NaN,NaN,51,NaN,NaN,NaN
129845,WAS201809260,20180926,0,Wednesday,405,F,D,WAS11,whitc901,millb901,...,165,8,suerw002,chenw001,NaN,NaN,42,NaN,NaN,NaN


In [59]:
home_team_game = game[['game_id'] + home_cols]
away_team_game = game[['game_id'] + away_cols]

In [60]:
home_team_game.shape, away_team_game.shape

((129846, 36), (129846, 36))

In [61]:
# create a column of True and a column of False, with appropriate indexes
s_true = game['game_id'] == game['game_id']
s_false = game['game_id'] != game['game_id']

In [62]:
# add the home team column
home_team_game = home_team_game.assign(home = s_true)
away_team_game = away_team_game.assign(home = s_false)

In [63]:
# rename the columns by removing 'home_'
home_column_mapping = {col:col[5:] for col in home_cols}
home_team_game = home_team_game.rename(columns = home_column_mapping)
home_team_game = bb.order_cols(home_team_game, ['game_id', 'home'])

# rename the columns by removing 'away_'
away_column_mapping = {col:col[5:] for col in away_cols}
away_team_game = away_team_game.rename(columns = away_column_mapping)
away_team_game = bb.order_cols(away_team_game, ['game_id', 'home'])

In [64]:
home_team_game.tail(3)

,game_id,home,team_id,start_pit_id,score_ct,hits_ct,err_ct,lob_ct,finish_pit_id,team_league_id,...,pitcher_ct,er_ct,ter_ct,wp_ct,bk_ct,po_ct,a_ct,pb_ct,dp_ct,tp_ct
129843,WAS201809240,True,WAS,stras001,7,9,2,8,dools001,N,...,5,2,2,1,0,27,10,0,0,0
129844,WAS201809250,True,WAS,schem001,9,11,2,9,cordj001,N,...,4,3,3,0,0,27,3,0,0,0
129845,WAS201809260,True,WAS,mcgok002,9,12,0,8,glovk001,N,...,5,3,3,0,0,21,6,0,0,0


In [65]:
away_team_game.tail(3)

,game_id,home,team_id,start_pit_id,score_ct,hits_ct,err_ct,lob_ct,finish_pit_id,team_league_id,...,pitcher_ct,er_ct,ter_ct,wp_ct,bk_ct,po_ct,a_ct,pb_ct,dp_ct,tp_ct
129843,WAS201809240,False,MIA,alcas001,3,6,0,10,gravb001,N,...,3,7,7,0,0,24,6,0,0,0
129844,WAS201809250,False,MIA,brigj002,4,9,1,6,meyeb002,N,...,5,9,9,1,0,24,13,0,3,0
129845,WAS201809260,False,MIA,chenw001,3,5,1,6,herne002,N,...,4,8,8,1,0,21,7,0,1,0


In [66]:
# concatenate the home and away dataframes
team_game = pd.concat([home_team_game, away_team_game])

In [67]:
# view a game
team_game[team_game['game_id'] == 'WAS201809260']

,game_id,home,team_id,start_pit_id,score_ct,hits_ct,err_ct,lob_ct,finish_pit_id,team_league_id,...,pitcher_ct,er_ct,ter_ct,wp_ct,bk_ct,po_ct,a_ct,pb_ct,dp_ct,tp_ct
129845,WAS201809260,True,WAS,mcgok002,9,12,0,8,glovk001,N,...,5,3,3,0,0,21,6,0,0,0
129845,WAS201809260,False,MIA,chenw001,3,5,1,6,herne002,N,...,4,8,8,1,0,21,7,0,1,0


In [68]:
# improve column names
names = {col:col.replace('_ct','') for col in team_game.columns if col.endswith('_ct')}
names

{'score_ct': 'score',
 'hits_ct': 'hits',
 'err_ct': 'err',
 'lob_ct': 'lob',
 'team_game_ct': 'team_game',
 'ab_ct': 'ab',
 '2b_ct': '2b',
 '3b_ct': '3b',
 'hr_ct': 'hr',
 'bi_ct': 'bi',
 'sh_ct': 'sh',
 'sf_ct': 'sf',
 'hp_ct': 'hp',
 'bb_ct': 'bb',
 'ibb_ct': 'ibb',
 'so_ct': 'so',
 'sb_ct': 'sb',
 'cs_ct': 'cs',
 'gdp_ct': 'gdp',
 'xi_ct': 'xi',
 'pitcher_ct': 'pitcher',
 'er_ct': 'er',
 'ter_ct': 'ter',
 'wp_ct': 'wp',
 'bk_ct': 'bk',
 'po_ct': 'po',
 'a_ct': 'a',
 'pb_ct': 'pb',
 'dp_ct': 'dp',
 'tp_ct': 'tp'}

In [69]:
# pitcher_ct actualy is a good name, it is the number of pitchers used in the game
names.pop('pitcher_ct')

'pitcher'

In [70]:
team_game = team_game.rename(columns=names)
team_game.columns

Index(['game_id', 'home', 'team_id', 'start_pit_id', 'score', 'hits', 'err',
       'lob', 'finish_pit_id', 'team_league_id', 'team_game', 'line_tx', 'ab',
       '2b', '3b', 'hr', 'bi', 'sh', 'sf', 'hp', 'bb', 'ibb', 'so', 'sb', 'cs',
       'gdp', 'xi', 'pitcher_ct', 'er', 'ter', 'wp', 'bk', 'po', 'a', 'pb',
       'dp', 'tp'],
      dtype='object')

In [71]:
names2 = {'2b':'b_2b', '3b':'b_3b'}

In [72]:
team_game = team_game.rename(columns=names2)
team_game.columns

Index(['game_id', 'home', 'team_id', 'start_pit_id', 'score', 'hits', 'err',
       'lob', 'finish_pit_id', 'team_league_id', 'team_game', 'line_tx', 'ab',
       'b_2b', 'b_3b', 'hr', 'bi', 'sh', 'sf', 'hp', 'bb', 'ibb', 'so', 'sb',
       'cs', 'gdp', 'xi', 'pitcher_ct', 'er', 'ter', 'wp', 'bk', 'po', 'a',
       'pb', 'dp', 'tp'],
      dtype='object')

In [73]:
team_game.isna().mean()

game_id           0.000000
home              0.000000
team_id           0.000000
start_pit_id      0.000000
score             0.000000
hits              0.000000
err               0.000000
lob               0.000000
finish_pit_id     0.130836
team_league_id    0.000000
team_game         1.000000
line_tx           0.000000
ab                0.000000
b_2b              0.000000
b_3b              0.000000
hr                0.000000
bi                0.000000
sh                0.000000
sf                0.000000
hp                0.000000
bb                0.000000
ibb               0.000000
so                0.000000
sb                0.000000
cs                0.000000
gdp               0.000000
xi                0.000000
pitcher_ct        0.000000
er                0.000000
ter               0.000000
wp                0.000000
bk                0.000000
po                0.000000
a                 0.000000
pb                0.000000
dp                0.000000
tp                0.000000
d

In [74]:
# team_game attribute is 100% null, drop it
team_game = team_game.drop('team_game', axis=1)

In [75]:
# game_id, home is unique
bb.is_unique(team_game, ['game_id', 'home'])

True

In [76]:
# game_id, team_id is unique
bb.is_unique(team_game, ['game_id', 'team_id'])

True

### Create pd.datetime from game_dt and start_game_tm

Here is the relevant information.
1. am/pm is not specified.
2. The time is not in 24-hour format.
3. The game_dt and start_game_tm are represented as integers.
4. A start_game_time value of zero means the start time is unknown.
5. The daynight_park_cd is never missing.  It is usually, but not always, correct.  This specifies whether the game is a "day game" or a "night game".
6. MLB domain knowledge: Some games may start late, due to a rain delay for example, but games never start after midnight.
7. MLB domain knowledge: Some games may start early, to allow for travel to the next city, but games never start before 9 am.

Given the above, am/pm can be deduced as follows:
* start_game_tm == 0 => use midnight (to represent unknown time)
* start_game_tm >= 1200 => pm
* start_game_tm < 900 => pm
* 900 <= start_game_tm < 1200, and day/night = day, => am
* 900 <= start_game_tm < 1200, and day/night = night, => pm

In [77]:
def parse_datetime(row):
    date = row['game_dt']
    time = row['start_game_tm']
    day_night = row['daynight_park_cd']
    
    if time > 0 and time < 900:
        time += 1200
    elif (900 <= time < 1200) and day_night == 'N':
        time += 1200

    time_str = f'{time//100:02d}:{time%100:02d}'
    datetime_str = str(date) + ' ' + time_str
    return pd.to_datetime(datetime_str, format='%Y%m%d %H:%M')

In [78]:
# create new datetime column
game_tidy['game_date'] = game_tidy.apply(parse_datetime, axis=1)

In [79]:
# these are no longer necessary
game_tidy = game_tidy.drop(['game_dt', 'game_dy', 'start_game_tm'], axis=1)

In [80]:
game_tidy.columns

Index(['game_id', 'game_ct', 'dh_fl', 'daynight_park_cd', 'park_id',
       'base4_ump_id', 'base1_ump_id', 'base2_ump_id', 'base3_ump_id',
       'lf_ump_id', 'rf_ump_id', 'attend_park_ct', 'scorer_record_id',
       'translator_record_id', 'inputter_record_id', 'input_record_ts',
       'edit_record_ts', 'method_record_cd', 'pitches_record_cd',
       'temp_park_ct', 'wind_direction_park_cd', 'wind_speed_park_ct',
       'field_park_cd', 'precip_park_cd', 'sky_park_cd', 'minutes_game_ct',
       'inn_ct', 'win_pit_id', 'lose_pit_id', 'save_pit_id', 'gwrbi_bat_id',
       'outs_ct', 'completion_tx', 'forfeit_tx', 'protest_tx', 'game_date'],
      dtype='object')

### Remove Fields Not Used in (Later) Analysis

In [81]:
ump_cols = [col for col in game_tidy.columns if 'ump' in col]
ump_cols

['base4_ump_id',
 'base1_ump_id',
 'base2_ump_id',
 'base3_ump_id',
 'lf_ump_id',
 'rf_ump_id']

In [82]:
record_cols = [col for col in game_tidy.columns if 'record' in col]
record_cols

['scorer_record_id',
 'translator_record_id',
 'inputter_record_id',
 'input_record_ts',
 'edit_record_ts',
 'method_record_cd',
 'pitches_record_cd']

In [83]:
tx_cols = [col for col in game_tidy.columns if 'tx' in col]
tx_cols

['completion_tx', 'forfeit_tx', 'protest_tx']

In [84]:
# remove these columns
drop_cols = ump_cols + record_cols + tx_cols
game_tidy = game_tidy.drop(drop_cols, axis=1)

In [85]:
game_tidy.columns

Index(['game_id', 'game_ct', 'dh_fl', 'daynight_park_cd', 'park_id',
       'attend_park_ct', 'temp_park_ct', 'wind_direction_park_cd',
       'wind_speed_park_ct', 'field_park_cd', 'precip_park_cd', 'sky_park_cd',
       'minutes_game_ct', 'inn_ct', 'win_pit_id', 'lose_pit_id', 'save_pit_id',
       'gwrbi_bat_id', 'outs_ct', 'game_date'],
      dtype='object')

### Decode Park Field Values
See: http://chadwick.sourceforge.net/doc/cwgame.html

In [86]:
park_fields = [col for col in game_tidy.columns if 'park' in col]
park_fields

['daynight_park_cd',
 'park_id',
 'attend_park_ct',
 'temp_park_ct',
 'wind_direction_park_cd',
 'wind_speed_park_ct',
 'field_park_cd',
 'precip_park_cd',
 'sky_park_cd']

In [87]:
park = game_tidy[park_fields].copy()
park.nunique()

daynight_park_cd              2
park_id                      79
attend_park_ct            44892
temp_park_ct                 90
wind_direction_park_cd        9
wind_speed_park_ct           51
field_park_cd                 5
precip_park_cd                6
sky_park_cd                   6
dtype: int64

In [88]:
# no nulls represented as np.nan
park.isna().sum()

daynight_park_cd          0
park_id                   0
attend_park_ct            0
temp_park_ct              0
wind_direction_park_cd    0
wind_speed_park_ct        0
field_park_cd             0
precip_park_cd            0
sky_park_cd               0
dtype: int64

In [89]:
# Day or Night Game
park['daynight_park_cd'].value_counts()

N    83738
D    46108
Name: daynight_park_cd, dtype: int64

In [90]:
# park id for Park table below
park['park_id'].value_counts().head(3)

CHI11    5101
BOS07    5097
LOS03    4877
Name: park_id, dtype: int64

In [91]:
# attendance
park['attend_park_ct'].value_counts().sort_index().head()

-1         1
 0      5009
 306       1
 365       1
 409       1
Name: attend_park_ct, dtype: int64

In [92]:
# attendance can never be 0 or -1, make these na
park['attend_park_ct'] = park['attend_park_ct'].replace([0, -1], np.nan)

In [93]:
# attendance
park['attend_park_ct'].value_counts().sort_index().head()

306.0    1
365.0    1
409.0    1
413.0    1
461.0    1
Name: attend_park_ct, dtype: int64

In [94]:
# note: there was one baseball game in which the public was not allowed to attend
# this is arguably null, as people wanted to attend, but could not
# click on the following generated link to read about it
bb.game_id_to_url('BAL201504290')

In [95]:
park['temp_park_ct'].value_counts().sort_index().head()

-1        85
 0     49108
 12        1
 14        1
 19        1
Name: temp_park_ct, dtype: int64

In [96]:
# temperature can never be 0 or -1 F, for an MLB game
park['temp_park_ct'] = park['temp_park_ct'].replace([0, -1], np.nan)

In [97]:
park['temp_park_ct'].value_counts().sort_index().head()

12.0    1
14.0    1
19.0    1
20.0    1
23.0    1
Name: temp_park_ct, dtype: int64

In [98]:
park['wind_direction_park_cd'].value_counts()

0    70961
4    12609
8    10184
2     8586
3     7509
1     6816
5     4907
6     4637
7     3637
Name: wind_direction_park_cd, dtype: int64

In [99]:
# http://chadwick.sourceforge.net/doc/cwgame.html#cwtools-cwgame-winddirection
direction = {
    0:'unknown',
    1:'to_lf',
    2:'to_cf',
    3:'to_rf',
    4:'l_to_r',
    5:'from_lf',
    6:'from_cf',
    7:'from_rf',
    8:'r_to_l'}

In [100]:
park['wind_direction_park_cd'] = park['wind_direction_park_cd'].map(direction)
park['wind_direction_park_cd'].value_counts()

unknown    70961
l_to_r     12609
r_to_l     10184
to_cf       8586
to_rf       7509
to_lf       6816
from_lf     4907
from_cf     4637
from_rf     3637
Name: wind_direction_park_cd, dtype: int64

In [101]:
park['wind_direction_park_cd'] = park['wind_direction_park_cd'].replace('unknown', np.nan)
park['wind_direction_park_cd'].value_counts(dropna=False)

NaN        70961
l_to_r     12609
r_to_l     10184
to_cf       8586
to_rf       7509
to_lf       6816
from_lf     4907
from_cf     4637
from_rf     3637
Name: wind_direction_park_cd, dtype: int64

In [102]:
park['wind_speed_park_ct'].value_counts().sort_index().head()

-1    59819
 0     9835
 1      816
 2     1690
 3     2563
Name: wind_speed_park_ct, dtype: int64

In [103]:
park['wind_speed_park_ct'] = park['wind_speed_park_ct'].replace(-1, np.nan)
park['wind_speed_park_ct'].isna().sum()

59819

In [104]:
park['wind_speed_park_ct'].value_counts().sort_index().head()

0.0    9835
1.0     816
2.0    1690
3.0    2563
4.0    2293
Name: wind_speed_park_ct, dtype: int64

In [105]:
# http://chadwick.sourceforge.net/doc/cwgame.html#cwtools-cwgame-fieldcondition
condition = {
    0:'unknown',
    1:'soaked',
    2:'wet',
    3:'damp',
    4:'dry'}

In [106]:
park['field_park_cd'] = park['field_park_cd'].map(condition)
park['field_park_cd'].value_counts()

unknown    103217
dry         23367
wet          2470
damp          529
soaked        263
Name: field_park_cd, dtype: int64

In [107]:
park['field_park_cd'] = park['field_park_cd'].replace('unknown', np.nan)
park['field_park_cd'].value_counts(dropna=False)

NaN       103217
dry        23367
wet         2470
damp         529
soaked       263
Name: field_park_cd, dtype: int64

In [108]:
# http://chadwick.sourceforge.net/doc/cwgame.html#cwtools-cwgame-precipitation
precip = {
    0:'unknown',
    1:'none',
    2:'drizzle',
    3:'showers',
    4:'rain',
    5:'snow'}

In [109]:
park['precip_park_cd'] = park['precip_park_cd'].map(precip)
park['precip_park_cd'].value_counts()

unknown    94704
none       32221
rain        1590
drizzle      868
showers      439
snow          24
Name: precip_park_cd, dtype: int64

In [110]:
park['precip_park_cd'] = park['precip_park_cd'].replace('unknown', np.nan)
park['precip_park_cd'].value_counts(dropna=False)

NaN        94704
none       32221
rain        1590
drizzle      868
showers      439
snow          24
Name: precip_park_cd, dtype: int64

In [111]:
# http://chadwick.sourceforge.net/doc/cwgame.html#cwtools-cwgame-sky
sky = {
    0:'unknown',
    1:'sunny',
    2:'cloudy',
    3:'overcast',
    4:'night',
    5:'dome'}

In [112]:
park['sky_park_cd'] = park['sky_park_cd'].map(sky)
park['sky_park_cd'].value_counts()

unknown     50417
sunny       21776
night       20833
cloudy      20154
dome        11584
overcast     5082
Name: sky_park_cd, dtype: int64

In [113]:
park['sky_park_cd'] = park['sky_park_cd'].replace('unknown', np.nan)
park['sky_park_cd'].value_counts(dropna=False)

NaN         50417
sunny       21776
night       20833
cloudy      20154
dome        11584
overcast     5082
Name: sky_park_cd, dtype: int64

In [114]:
# null is now represented as np.nan
park.isna().sum()

daynight_park_cd               0
park_id                        0
attend_park_ct              5010
temp_park_ct               49193
wind_direction_park_cd     70961
wind_speed_park_ct         59819
field_park_cd             103217
precip_park_cd             94704
sky_park_cd                50417
dtype: int64

In [115]:
# copy the wrangled park data back to game_tidy
game_tidy[park.columns] = park

In [116]:
game_tidy.columns

Index(['game_id', 'game_ct', 'dh_fl', 'daynight_park_cd', 'park_id',
       'attend_park_ct', 'temp_park_ct', 'wind_direction_park_cd',
       'wind_speed_park_ct', 'field_park_cd', 'precip_park_cd', 'sky_park_cd',
       'minutes_game_ct', 'inn_ct', 'win_pit_id', 'lose_pit_id', 'save_pit_id',
       'gwrbi_bat_id', 'outs_ct', 'game_date'],
      dtype='object')

In [117]:
game_tidy = bb.order_cols(game_tidy, ['game_id', 'game_date'])
game_tidy.tail(3)

,game_id,game_date,game_ct,dh_fl,daynight_park_cd,park_id,attend_park_ct,temp_park_ct,wind_direction_park_cd,wind_speed_park_ct,field_park_cd,precip_park_cd,sky_park_cd,minutes_game_ct,inn_ct,win_pit_id,lose_pit_id,save_pit_id,gwrbi_bat_id,outs_ct
129843,WAS201809240,2018-09-24 19:06:00,0,F,N,WAS11,22428.0,63.0,from_rf,10.0,wet,rain,NaN,189,9,millj006,alcas001,NaN,NaN,51
129844,WAS201809250,2018-09-25 19:05:00,0,F,N,WAS11,26483.0,78.0,to_lf,4.0,NaN,NaN,cloudy,192,9,schem001,brigj002,NaN,NaN,51
129845,WAS201809260,2018-09-26 16:05:00,0,F,D,WAS11,28680.0,88.0,r_to_l,6.0,NaN,NaN,NaN,165,8,suerw002,chenw001,NaN,NaN,42


In [118]:
# the primary key is (game_id), verify no dups
game_tidy['game_id'].is_unique

True

### Add Year, Lahman Team Id and Game Date to Team Game
This will make later queries easier.

In [119]:
team_game.shape

(259692, 36)

In [120]:
# add game_date
team_game = pd.merge(team_game, game_tidy[['game_id','game_date']],on = 'game_id')

In [121]:
# add game year
team_game['year_id'] = team_game['game_date'].dt.year

In [122]:
team_game.shape

(259692, 38)

In [123]:
# add lahman team_id from teams
team_game = pd.merge(team_game, teams[['team_id', 'year_id', 'team_id_retro']],
                     left_on = ['year_id', 'team_id'], 
                     right_on = ['year_id', 'team_id_retro'],
                     suffixes=['', '_lahman'])

In [124]:
team_game = team_game.drop('team_id_retro', axis= 1)
team_game.shape

(259692, 39)

In [125]:
team_game = bb.order_cols(team_game, ['game_id', 'game_date', 
                                      'year_id', 'team_id', 
                                      'team_id_lahman'])
team_game.tail(3)

,game_id,game_date,year_id,team_id,team_id_lahman,home,start_pit_id,score,hits,err,...,pitcher_ct,er,ter,wp,bk,po,a,pb,dp,tp
259689,WAS201809081,2018-09-08 17:15:00,2018,CHN,CHN,False,garcj004,3,9,3,...,6,7,7,2,0,24,17,0,2,0
259690,WAS201809082,2018-09-08 21:00:00,2018,CHN,CHN,False,hamec001,5,6,0,...,5,6,6,0,0,24,4,0,1,0
259691,WAS201809130,2018-09-13 16:05:00,2018,CHN,CHN,False,montm002,4,10,0,...,9,3,3,2,0,30,6,0,0,0


In [126]:
# Note: 97% of the time, these are the same
(team_game.team_id == team_game.team_id_lahman).mean()

0.9677849144371025

In [127]:
os.chdir(p_wrangled)
%time bb.to_csv_with_types(team_game, 'team_game.csv.gz')

CPU times: user 15.6 s, sys: 7.97 ms, total: 15.6 s
Wall time: 15.6 s


In [128]:
os.chdir(p_wrangled)
%time bb.to_csv_with_types(game_tidy, 'game.csv.gz')

CPU times: user 2.25 s, sys: 61 µs, total: 2.25 s
Wall time: 2.25 s


## 3. Scrape Data for Players Lookup Table

player_id to player information is needed.

This data is similar to Lahman's People.csv data, but it has different fields.

There is no separate file for this.  It will be scraped from a web page.

In [129]:
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

In [130]:
# get the web page
r = requests.get("https://www.retrosheet.org/retroID.htm")
soup = BeautifulSoup(r.content, 'lxml')

# data is within the pre tag
table_txt = soup.pre.string

# remove unnecessary double quotes
table_txt = table_txt.replace('"','')

In [131]:
# read from this string instead of file
players = pd.read_csv(StringIO(table_txt), 
    parse_dates=['Play debut', 'Mgr debut', 'Ump debut'])

In [132]:
# Coach debut has some bad values
def parse_date(value):
    # perhaps 43188 means 04/31/1988, but use null as unsure
    # no coach debuted prior to the year 1800
    if pd.isna(value) or value == '43188' or int(value[-4:]) < 1800:
        return pd.NaT
    else:
        return pd.datetime.strptime(value, '%m/%d/%Y')

In [133]:
players['Coach debut'] = players['Coach debut'].apply(parse_date)
players.head()

,ID,Last,First,Play debut,Mgr debut,Coach debut,Ump debut
0,aardd001,Aardsma,David,2004-04-06,NaT,NaT,NaT
1,aaroh101,Aaron,Hank,1954-04-13,NaT,NaT,NaT
2,aarot101,Aaron,Tommie,1962-04-10,NaT,1979-04-06,NaT
3,aased001,Aase,Don,1977-07-26,NaT,NaT,NaT
4,abada001,Abad,Andy,2001-09-10,NaT,NaT,NaT


In [134]:
name_chg = {'ID':'player_id',
         'Last':'last_name',
         'First':'first_name',
         'Play debut':'player_debut',
         'Mgr debut':'mgr_debut',
         'Coach debut': 'coach_debut',
         'Ump debut':'ump_debut'}
players = players.rename(columns=name_chg)
players.head()

,player_id,last_name,first_name,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,2004-04-06,NaT,NaT,NaT
1,aaroh101,Aaron,Hank,1954-04-13,NaT,NaT,NaT
2,aarot101,Aaron,Tommie,1962-04-10,NaT,1979-04-06,NaT
3,aased001,Aase,Don,1977-07-26,NaT,NaT,NaT
4,abada001,Abad,Andy,2001-09-10,NaT,NaT,NaT


#### Persist Players

In [135]:
os.chdir(p_wrangled)
bb.to_csv_with_types(players, 'players.csv')

## 4. Scrape Data for Stadium (Park) Lookup Table

park_id to park information is needed.  

This data is similar to Lahman's Parks.csv, but has different fields.

There is no separate file for this.  It will be scraped from a web page.

In [136]:
# get the web page (this is not html!)
r = requests.get("https://www.retrosheet.org/parkcode.txt")

table_txt = r.content.decode("utf-8")

# read from this string instead of file
parks = pd.read_csv(StringIO(table_txt), parse_dates=['START', 'END'])

In [137]:
parks.columns = parks.columns.str.lower()
parks = parks.rename(columns={'parkid':'park_id','name':'park_name',
                              'start':'part_start','end':'park_end'})
parks.head()

,park_id,park_name,aka,city,state,part_start,park_end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,1880-09-11,1882-05-30,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,1884-04-30,1884-05-31,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,1966-04-19,NaT,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,1972-04-21,1993-10-03,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,1994-04-11,NaT,AL,NaN


#### Persist Stadiums

In [138]:
os.chdir(p_wrangled)
bb.to_csv_with_types(parks, 'parks.csv')